# AgroGraphNet: Feature Engineering

This notebook creates comprehensive features for machine learning including node features, edge features, and temporal patterns.

## Objectives:
1. Create node features from satellite, weather, and farm data
2. Engineer edge features based on spatial and environmental relationships
3. Create temporal features and sequences
4. Prepare feature matrices for GNN training
5. Feature selection and importance analysis

In [1]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import torch
from torch_geometric.data import Data
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from config import *
from data_utils import *
from graph_utils import *
from visualization import *

# Set random seed for reproducibility
np.random.seed(RANDOM_SEED)

print("Libraries imported successfully!")
print(f"Loading processed data from: {PROCESSED_DATA_DIR}")

Libraries imported successfully!
Loading processed data from: C:\Users\Ash09\agro_tech\AgroGraphNet\notebooks\..\data\processed


## 1. Load Processed Data

In [2]:
# Load processed data from previous notebooks
print("Loading processed datasets...")

# Load feature matrix
features_file = PROCESSED_DATA_DIR / 'features_scaled.csv'
if features_file.exists():
    features_df = pd.read_csv(features_file)
    features_df['date'] = pd.to_datetime(features_df['date'])
    print(f"✅ Loaded feature matrix: {features_df.shape}")
else:
    raise FileNotFoundError("Processed features not found. Please run notebooks 01-03 first.")

# Load farm locations
farm_files = list(FARM_LOCATIONS_DIR.glob('*.csv'))
if farm_files:
    farms_df = pd.read_csv(farm_files[0])
    print(f"✅ Loaded farm locations: {len(farms_df)} farms")
else:
    raise FileNotFoundError("Farm locations not found.")

# Load graph data if available
graph_file = GRAPHS_DIR / 'farm_graphs.pkl'
if graph_file.exists():
    import pickle
    with open(graph_file, 'rb') as f:
        graph_data = pickle.load(f)
    print(f"✅ Loaded graph data: {len(graph_data)} time steps")
else:
    print("⚠️ Graph data not found. Will create graphs in this notebook.")
    graph_data = None

print(f"\nData overview:")
print(f"- Features shape: {features_df.shape}")
print(f"- Time points: {features_df['date'].nunique()}")
print(f"- Unique farms: {features_df['farm_id'].nunique()}")
print(f"- Disease classes: {features_df['disease_type'].nunique()}")

Loading processed datasets...
✅ Loaded feature matrix: (1200, 37)
✅ Loaded farm locations: 100 farms
⚠️ Graph data not found. Will create graphs in this notebook.

Data overview:
- Features shape: (1200, 37)
- Time points: 12
- Unique farms: 100
- Disease classes: 5


## 2. Node Feature Engineering

In [3]:
# Create comprehensive node features
print("Engineering node features...")

# Identify feature columns
exclude_cols = ['farm_id', 'date', 'disease_type', 'disease_label', 'severity', 'is_diseased']
feature_columns = [col for col in features_df.columns if col not in exclude_cols]

print(f"Base feature columns: {len(feature_columns)}")

# Create additional engineered features
features_engineered = features_df.copy()

# 1. Vegetation health indicators
if 'vegetation_NDVI' in features_df.columns and 'vegetation_EVI' in features_df.columns:
    features_engineered['vegetation_health_score'] = (
        features_df['vegetation_NDVI'] + features_df['vegetation_EVI']
    ) / 2
    print("✅ Created vegetation health score")

# 2. Weather stress indicators
weather_cols = [col for col in features_df.columns if col.startswith('weather_')]
if len(weather_cols) > 0:
    # Temperature stress (deviation from optimal)
    if 'weather_temperature' in features_df.columns:
        optimal_temp = 20  # Celsius
        features_engineered['temperature_stress'] = np.abs(
            features_df['weather_temperature'] - optimal_temp
        )
    
    # Humidity stress
    if 'weather_humidity' in features_df.columns:
        optimal_humidity = 60  # Percentage
        features_engineered['humidity_stress'] = np.abs(
            features_df['weather_humidity'] - optimal_humidity
        )
    
    # Weather variability (if rolling features exist)
    rolling_std_cols = [col for col in features_df.columns if 'rolling_std' in col]
    if rolling_std_cols:
        features_engineered['weather_variability'] = features_df[rolling_std_cols].mean(axis=1)
    
    print("✅ Created weather stress indicators")

# 3. Seasonal features
if 'month' in features_df.columns:
    # Cyclical encoding for month
    features_engineered['month_sin'] = np.sin(2 * np.pi * features_df['month'] / 12)
    features_engineered['month_cos'] = np.cos(2 * np.pi * features_df['month'] / 12)
    print("✅ Created cyclical month features")

# 4. Farm size categories
if 'area_hectares' in features_df.columns:
    area_quartiles = features_df['area_hectares'].quantile([0.25, 0.5, 0.75])
    features_engineered['farm_size_category'] = pd.cut(
        features_df['area_hectares'],
        bins=[-np.inf, area_quartiles[0.25], area_quartiles[0.5], area_quartiles[0.75], np.inf],
        labels=['Small', 'Medium', 'Large', 'Very Large']
    )
    
    # One-hot encode farm size categories
    size_dummies = pd.get_dummies(features_engineered['farm_size_category'], prefix='size')
    features_engineered = pd.concat([features_engineered, size_dummies], axis=1)
    features_engineered.drop('farm_size_category', axis=1, inplace=True)
    print("✅ Created farm size categories")

# 5. Historical disease features (lag features)
print("Creating historical disease features...")
features_engineered = features_engineered.sort_values(['farm_id', 'date'])

# Previous disease status (1 time step lag)
features_engineered['prev_disease_label'] = features_engineered.groupby('farm_id')['disease_label'].shift(1)
features_engineered['prev_severity'] = features_engineered.groupby('farm_id')['severity'].shift(1)
features_engineered['prev_is_diseased'] = features_engineered.groupby('farm_id')['is_diseased'].shift(1)

# Fill NaN values for first time step
features_engineered['prev_disease_label'].fillna(0, inplace=True)  # Assume healthy initially
features_engineered['prev_severity'].fillna(0, inplace=True)
features_engineered['prev_is_diseased'].fillna(0, inplace=True)

# Disease history (cumulative)
features_engineered['disease_history_count'] = features_engineered.groupby('farm_id')['is_diseased'].cumsum()
features_engineered['avg_historical_severity'] = features_engineered.groupby('farm_id')['severity'].expanding().mean().reset_index(0, drop=True)

print("✅ Created historical disease features")

# Update feature columns list
new_feature_columns = [col for col in features_engineered.columns if col not in exclude_cols]
print(f"\nTotal engineered features: {len(new_feature_columns)}")
print(f"Added {len(new_feature_columns) - len(feature_columns)} new features")

Engineering node features...
Base feature columns: 31
✅ Created vegetation health score
✅ Created weather stress indicators
✅ Created cyclical month features
✅ Created farm size categories
Creating historical disease features...
✅ Created historical disease features

Total engineered features: 46
Added 15 new features


## 3. Edge Feature Engineering

In [4]:
# Create edge features for graph construction
print("Engineering edge features...")

# Calculate distance matrix
distance_matrix = create_distance_matrix(farms_df)
print(f"✅ Distance matrix created: {distance_matrix.shape}")

# Calculate environmental similarity for each time point
time_points = sorted(features_engineered['date'].unique())
edge_features_by_time = {}

for time_point in time_points:
    print(f"Processing time point: {time_point}")
    
    # Get data for this time point
    time_data = features_engineered[features_engineered['date'] == time_point].copy()
    time_data = time_data.sort_values('farm_id').reset_index(drop=True)
    
    # Environmental similarity based on weather features
    weather_features = [col for col in time_data.columns if col.startswith('weather_')]
    if weather_features:
        weather_data = time_data[weather_features].values
        
        # Calculate pairwise similarities
        from sklearn.metrics.pairwise import cosine_similarity
        env_similarity = cosine_similarity(weather_data)
    else:
        env_similarity = np.ones((len(time_data), len(time_data)))
    
    # Vegetation similarity
    vegetation_features = [col for col in time_data.columns if col.startswith('vegetation_')]
    if vegetation_features:
        vegetation_data = time_data[vegetation_features].values
        vegetation_similarity = cosine_similarity(vegetation_data)
    else:
        vegetation_similarity = np.ones((len(time_data), len(time_data)))
    
    # Crop type similarity
    crop_features = [col for col in time_data.columns if col.startswith('crop_')]
    if crop_features:
        crop_data = time_data[crop_features].values
        crop_similarity = cosine_similarity(crop_data)
    else:
        crop_similarity = np.ones((len(time_data), len(time_data)))
    
    # Combine edge features
    edge_features = {
        'distance': distance_matrix,
        'environmental_similarity': env_similarity,
        'vegetation_similarity': vegetation_similarity,
        'crop_similarity': crop_similarity,
        'combined_similarity': (env_similarity + vegetation_similarity + crop_similarity) / 3
    }
    
    edge_features_by_time[time_point] = edge_features

print(f"✅ Edge features created for {len(time_points)} time points")
print(f"Edge feature types: {list(edge_features.keys())}")

Engineering edge features...
✅ Distance matrix created: (100, 100)
Processing time point: 2023-01-31 00:00:00
Processing time point: 2023-02-28 00:00:00
Processing time point: 2023-03-31 00:00:00
Processing time point: 2023-04-30 00:00:00
Processing time point: 2023-05-31 00:00:00
Processing time point: 2023-06-30 00:00:00
Processing time point: 2023-07-31 00:00:00
Processing time point: 2023-08-31 00:00:00
Processing time point: 2023-09-30 00:00:00
Processing time point: 2023-10-31 00:00:00
Processing time point: 2023-11-30 00:00:00
Processing time point: 2023-12-31 00:00:00
✅ Edge features created for 12 time points
Edge feature types: ['distance', 'environmental_similarity', 'vegetation_similarity', 'crop_similarity', 'combined_similarity']


## 4. Feature Selection and Importance Analysis

In [5]:
# Analyze feature importance for disease prediction
print("Analyzing feature importance...")

# Prepare data for feature selection
feature_cols = [col for col in features_engineered.columns if col not in exclude_cols]
X = features_engineered[feature_cols].fillna(0)
y = features_engineered['disease_label']

print(f"Feature matrix shape: {X.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")

# Remove constant features
constant_features = []
for col in X.columns:
    if X[col].nunique() <= 1:
        constant_features.append(col)

if constant_features:
    print(f"Removing {len(constant_features)} constant features")
    X = X.drop(columns=constant_features)

# Feature selection using ANOVA F-test
selector = SelectKBest(score_func=f_classif, k=min(50, X.shape[1]))  # Select top 50 features
X_selected = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()].tolist()
feature_scores = selector.scores_[selector.get_support()]

print(f"\n✅ Selected {len(selected_features)} most important features")

# Create feature importance DataFrame
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance_score': feature_scores
}).sort_values('importance_score', ascending=False)

print("\nTop 10 most important features:")
display(feature_importance.head(10))

# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(20)
plt.barh(range(len(top_features)), top_features['importance_score'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance Score (F-statistic)')
plt.title('Top 20 Feature Importance for Disease Prediction')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig(RESULTS_DIR / '04_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# Save feature importance
feature_importance.to_csv(PROCESSED_DATA_DIR / 'feature_importance.csv', index=False)
print(f"✅ Feature importance saved to {PROCESSED_DATA_DIR / 'feature_importance.csv'}")

Analyzing feature importance...
Feature matrix shape: (1200, 46)
Target distribution: {0: 867, 2: 113, 1: 99, 3: 75, 4: 46}


ValueError: could not convert string to float: 'Winter'

## 5. Create Final Feature Matrices

In [ ]:
# Create final feature matrices for GNN training
print("Creating final feature matrices...")

# Use selected features for final matrix
final_features = features_engineered[['farm_id', 'date'] + selected_features + 
                                   ['disease_type', 'disease_label', 'severity', 'is_diseased']].copy()

print(f"Final feature matrix shape: {final_features.shape}")

# Scale the selected features
scaler = StandardScaler()
final_features[selected_features] = scaler.fit_transform(final_features[selected_features])

print("✅ Features scaled using StandardScaler")

# Create node feature matrices for each time point
node_features_by_time = {}
labels_by_time = {}

for time_point in time_points:
    time_data = final_features[final_features['date'] == time_point].copy()
    time_data = time_data.sort_values('farm_id').reset_index(drop=True)
    
    # Node features (exclude metadata and labels)
    node_features = time_data[selected_features].values
    labels = time_data['disease_label'].values
    
    node_features_by_time[time_point] = node_features
    labels_by_time[time_point] = labels

print(f"✅ Node features created for {len(time_points)} time points")
print(f"Node feature dimension: {node_features.shape[1]}")

# Save processed features
final_features.to_csv(PROCESSED_DATA_DIR / 'final_features.csv', index=False)
print(f"✅ Final features saved to {PROCESSED_DATA_DIR / 'final_features.csv'}")

# Save scaler
import joblib
joblib.dump(scaler, PROCESSED_DATA_DIR / 'feature_scaler.pkl')
print(f"✅ Feature scaler saved to {PROCESSED_DATA_DIR / 'feature_scaler.pkl'}")

## 6. Temporal Feature Analysis

In [ ]:
# Analyze temporal patterns in features
print("Analyzing temporal patterns...")

# Calculate feature stability over time
feature_stability = {}

for feature in selected_features[:10]:  # Analyze top 10 features
    feature_values_by_time = []
    
    for time_point in time_points:
        time_data = final_features[final_features['date'] == time_point]
        feature_values_by_time.append(time_data[feature].mean())
    
    # Calculate coefficient of variation (stability measure)
    cv = np.std(feature_values_by_time) / (np.mean(feature_values_by_time) + 1e-8)
    feature_stability[feature] = cv

# Visualize temporal patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Disease progression over time
disease_progression = final_features.groupby('date')['is_diseased'].mean()
axes[0, 0].plot(disease_progression.index, disease_progression.values, 'o-', color='red')
axes[0, 0].set_title('Disease Prevalence Over Time')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Proportion of Diseased Farms')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Feature stability
stability_df = pd.DataFrame(list(feature_stability.items()), columns=['Feature', 'CV'])
stability_df = stability_df.sort_values('CV')
axes[0, 1].barh(range(len(stability_df)), stability_df['CV'])
axes[0, 1].set_yticks(range(len(stability_df)))
axes[0, 1].set_yticklabels(stability_df['Feature'])
axes[0, 1].set_xlabel('Coefficient of Variation')
axes[0, 1].set_title('Feature Temporal Stability')

# 3. Vegetation health over time
if 'vegetation_health_score' in final_features.columns:
    veg_health = final_features.groupby('date')['vegetation_health_score'].mean()
    axes[1, 0].plot(veg_health.index, veg_health.values, 'o-', color='green')
    axes[1, 0].set_title('Average Vegetation Health Over Time')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].set_ylabel('Vegetation Health Score')
    axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Weather stress over time
if 'temperature_stress' in final_features.columns:
    temp_stress = final_features.groupby('date')['temperature_stress'].mean()
    axes[1, 1].plot(temp_stress.index, temp_stress.values, 'o-', color='orange')
    axes[1, 1].set_title('Average Temperature Stress Over Time')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Temperature Stress')
    axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(RESULTS_DIR / '04_temporal_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Temporal analysis completed")

## 7. Save Engineered Features

In [ ]:
# Save all engineered data for next notebooks
print("Saving engineered features...")

# Save node features and labels by time
import pickle

feature_data = {
    'node_features_by_time': node_features_by_time,
    'labels_by_time': labels_by_time,
    'edge_features_by_time': edge_features_by_time,
    'selected_features': selected_features,
    'feature_importance': feature_importance,
    'time_points': time_points,
    'farms_df': farms_df,
    'distance_matrix': distance_matrix
}

with open(PROCESSED_DATA_DIR / 'engineered_features.pkl', 'wb') as f:
    pickle.dump(feature_data, f)

print(f"✅ Engineered features saved to {PROCESSED_DATA_DIR / 'engineered_features.pkl'}")

# Save summary statistics
summary_stats = {
    'total_features': len(selected_features),
    'total_farms': len(farms_df),
    'total_time_points': len(time_points),
    'disease_classes': len(final_features['disease_type'].unique()),
    'feature_dimension': node_features.shape[1],
    'total_samples': len(final_features)
}

with open(PROCESSED_DATA_DIR / 'feature_summary.json', 'w') as f:
    import json
    json.dump(summary_stats, f, indent=2)

print(f"✅ Summary statistics saved to {PROCESSED_DATA_DIR / 'feature_summary.json'}")

print("\n🎉 Feature engineering completed successfully!")
print("\nSummary:")
print(f"- Total engineered features: {len(selected_features)}")
print(f"- Node feature dimension: {node_features.shape[1]}")
print(f"- Time points: {len(time_points)}")
print(f"- Total samples: {len(final_features)}")
print(f"- Disease classes: {len(final_features['disease_type'].unique())}")

print("\nNext steps:")
print("1. Run notebook 05_model_development.ipynb")
print("2. Train GNN models using engineered features")
print("3. Compare with baseline models")